In [1]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

'C:\\Users\\daniel.martinezcarre\\.wdm\\drivers\\geckodriver\\win64\\v0.34.0\\geckodriver.exe'

In [2]:
import argparse
from enum import Enum
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
from bs4 import BeautifulSoup
import pprint
from io import StringIO
from bs4 import BeautifulSoup
import pandas as pd


In [3]:
def esperar_carga_pagina(driver, url):
    """
    Espera a que la página web se cargue completamente.

    Args:
        driver: El objeto del navegador web.
        url: La URL de la página web que se quiere cargar.
    """
    # Carga la página web
    driver.get(url)

    # Espera a que la página se cargue completamente
    wait = WebDriverWait(driver, 10)
    wait.until(lambda driver: driver.execute_script("return document.readyState") == "complete")


def automatizaciones(driver):
    """
    Ejecuta las automatizaciones en la página web con esperas.

    Args:
        driver: El objeto del navegador web.
    """
    pass


In [27]:
URL = "https://eumostwanted.eu/es"

In [71]:
driver = webdriver.Firefox()
esperar_carga_pagina(driver, URL)
# automatizaciones(driver)

In [72]:
def sacar_datos_criminal():
    row = {}
    body = driver.execute_script("return document.body")
    source = body.get_attribute('innerHTML')
    soup = BeautifulSoup(source, 'html.parser')

    try:
        nombre = soup.find("div", class_="field field--node-field-w-first-name field--name-field-w-first-name field--type-string field--label-hidden field__item")
        row["nome"] = nombre.text
    except:
        row["nome"] = "-"
        print("Non ten nome")

    try:
        pais_busqueda =  soup.find("div", class_="field field--node-field-enfast-country field--name-field-enfast-country field--type-entity-reference field--label-hidden field__item")
        row["pais_buscador"] = pais_busqueda.text.split(' ')[1]
    except:
        row["pais_buscador"] = "-"
        print("Non hay pais que o busque")

    try:
        data_nacemento = soup.find("div", class_="field field--node-field-date-of-birth field--name-field-date-of-birth field--type-datetime field--label-inline clearfix")
        fecha = data_nacemento.find("div", class_="field__item")
        row["data_nacemento"] = fecha.text.split('(')[0]
    except:
        row["data_nacemento"] = "-"
        print("Non hay data de nacemento")

    try:
        enl_nacionalidade = soup.find("div", class_="field field--node-field-nationality field--name-field-nationality field--type-entity-reference field--label-inline clearfix")
        nacionalidade = enl_nacionalidade.find("div", class_="field__items")
        row["nacionalidade"] = nacionalidade.text
    except:
        row["nacionalidade"] = "-"
        print("Non hay nacionalide")

    try:
        enl_etnia = soup.find("div", class_="field field--node-field-ethnic-origin field--name-field-ethnic-origin field--type-entity-reference field--label-inline clearfix")
        etnia = enl_etnia.find("div", class_="field__item")
        row["etnia"] = etnia.text
    except:
        row["etnia"] = "-"
        print("Non hay etnia")
    
    
    try:
        enl_idiomas = soup.find("div", class_="field field--node-field-languages field--name-field-languages field--type-entity-reference field--label-inline clearfix")
        num_lenguas = enl_idiomas.find("div", class_="field__items")
        num_lenguas = num_lenguas.find("div", class_="field__item")
        row["idiomas"] = nacionalidade.text
    except:
        row["idiomas"] = "-"
        print("Non hay idiomas")

    return row




### Carga de las urls de los criminales

In [73]:
def cargar_urls_criminales():
    lista = []
    divs = driver.find_elements(By.XPATH,'.//div[@class="wanted_list"]/div')
    for i in divs:
        enlace = i.get_attribute("href")
        lista.append(enlace)
    return lista

urls_criminales = cargar_urls_criminales()


In [74]:
criminales = []

for link in urls_criminales:
    if isinstance(link,str):
        url_carga = URL + "/#" + link
        print(url_carga)
    
        # Abrir enlace en pestaña nueva
        driver.execute_script(f"window.open('{url_carga}', 'new_window')")

        # Cambiar el foco a la nueva pestaña
        driver.switch_to.window(driver.window_handles[-1])

        # Esperar a que la página se cargue
        WebDriverWait(driver,1).until(
        EC.presence_of_element_located((By.CLASS_NAME, "off-canvas-content")))

        criminales.append(sacar_datos_criminal())
        print(criminales[-1])


        driver.close()

        # Cambiar el foco a la pestaña principal
        driver.switch_to.window(driver.window_handles[0])    

https://eumostwanted.eu/es/#/garweg-burkhard
{'nome': 'GARWEG, Burkhard', 'pais_buscador': 'Alemania', 'data_nacemento': 'Septiembre 1,  1968 ', 'nacionalidade': '\nGerman\n', 'etnia': 'European', 'idiomas': '\nGerman\n'}
https://eumostwanted.eu/es/#/staub-ernst-volker
{'nome': 'STAUB, Ernst-Volker', 'pais_buscador': 'Alemania', 'data_nacemento': 'Octubre 30,  1954 ', 'nacionalidade': '\nGerman\n', 'etnia': 'European', 'idiomas': '\nGerman\n'}
https://eumostwanted.eu/es/#/jankevics-maris
Non hay idiomas
{'nome': 'JANKEVICS, Māris', 'pais_buscador': 'Letonia', 'data_nacemento': 'Enero 24,  1979 ', 'nacionalidade': '\nLatvian\n', 'etnia': 'European', 'idiomas': '-'}
https://eumostwanted.eu/es/#/de-la-camara-guisado-ramon
{'nome': 'DE LA CAMARA GUISADO, Ramón', 'pais_buscador': 'España', 'data_nacemento': 'Agosto 29,  1952 ', 'nacionalidade': '\nSpanish\n', 'etnia': 'European', 'idiomas': '\nSpanish\n'}
https://eumostwanted.eu/es/#/qatra-amin
Non hay etnia
Non hay idiomas
{'nome': 'QATRA,

In [87]:
df_criminales = pd.DataFrame(criminales)
df_criminales["nacionalidade"] = df_criminales["nacionalidade"].str[1:-1]
df_criminales["idiomas"] = df_criminales["idiomas"].str[1:-1]
df_criminales[df_criminales["nacionalidade"] == ""] = "-"
df_criminales[df_criminales["idiomas"] == ""] = "-"
df_criminales

,nome,pais_buscador,data_nacemento,nacionalidade,etnia,idiomas
0,"GARWEG, Burkhard",Alemania,"Septiembre 1, 1968",German,European,German
1,"STAUB, Ernst-Volker",Alemania,"Octubre 30, 1954",German,European,German
2,-,-,-,-,-,-
3,"DE LA CAMARA GUISADO, Ramón",España,"Agosto 29, 1952",Spanish,European,Spanish
4,-,-,-,-,-,-
5,"VESKUS, MIHKEL",Estonia,"Septiembre 4, 1984",Estonian,-,Estonian
6,"CALLEJA MAATOUK, Jomic",Malta,"Noviembre 18, 1985",Maltese,European,Maltese
7,"OKOLIČANY, Róbert",Eslovaquia,"Septiembre 17, 1975",Slovak,-,Slovak
8,"DO, Steffen van Khoa",Dinamarca,"Marzo 30, 1988",Vietnamese,-,Vietnamese
9,-,-,-,-,-,-
